In [6]:
import pandas as pd

In [7]:
dataEvents = pd.read_csv('/Users/manuelgomezmoratilla/Desktop/TFG/data_processing_scripts/data/anonamyze_all_data_collection_v2.csv', sep=";")

In [8]:
import numpy as np
import json
import hashlib
from datetime import datetime
from datetime import timedelta
from collections import OrderedDict

pd.options.mode.chained_assignment = None  # default='warn'

def sequenceBetweenPuzzles(dataEvents, group = 'all'): 
    
    dataEvents['group'] = [json.loads(x)['group'] if 'group' in json.loads(x).keys() else '' for x in dataEvents['data']]
    dataEvents['user'] = [json.loads(x)['user'] if 'user' in json.loads(x).keys() else '' for x in dataEvents['data']]
    # removing those rows where we dont have a group and a user that is not guest
    dataEvents = dataEvents[((dataEvents['group'] != '') & (dataEvents['user'] != '') & (dataEvents['user'] != 'guest'))]
    dataEvents['group_user_id'] = dataEvents['group'] + '~' + dataEvents['user']

    # filtering to only take the group passed as argument
    if(group != 'all'):
        dataEvents = dataEvents[dataEvents['group'].isin(group)]
    
       # Data Cleaning
    dataEvents['time'] = pd.to_datetime(dataEvents['time'])
    dataEvents = dataEvents.sort_values('time')

    userPuzzleDict = {}

    for user in dataEvents['group_user_id'].unique():
            #Select rows
            user_events = dataEvents[dataEvents['group_user_id'] == user]
            user_events_na_dropped = user_events.dropna()
            for enum, event in user_events_na_dropped.iterrows():
                user_key = event['user']
                if(user_key not in userPuzzleDict.keys()):
                    userPuzzleDict[user_key] = {}
                    numPuzzles = 1
                if(event['type'] == 'ws-start_level'):
                    activePuzzle = json.loads(event['data'])['task_id']
                    if (activePuzzle == 'Sandbox'):
                        continue
                    secondKey = str(numPuzzles) + '~' + event['session_id']
                    if (userPuzzleDict[user_key].get(secondKey) == None):
                        userPuzzleDict[user_key][secondKey] = dict()
                    if (userPuzzleDict[user_key][secondKey].get(activePuzzle) == None):
                        userPuzzleDict[user_key][secondKey] = {activePuzzle : 'started'}
                elif(event['type'] == 'ws-puzzle_started'):
                    if (activePuzzle == 'Sandbox' or userPuzzleDict[user_key][secondKey][activePuzzle] in ['started','shape_created', 'submitted', 'completed']):
                        continue
                    userPuzzleDict[user_key][secondKey] = {activePuzzle : 'started'}
                elif(event['type'] == 'ws-create_shape'):
                    if (activePuzzle == 'Sandbox' or userPuzzleDict[user_key][secondKey][activePuzzle] in ['shape_created', 'submitted', 'completed']):
                        continue
                    userPuzzleDict[user_key][secondKey] = {activePuzzle : 'shape_created'}
                elif(event['type'] == 'ws-check_solution'):
                    if (activePuzzle == 'Sandbox' or userPuzzleDict[user_key][secondKey][activePuzzle] in ['submitted', 'completed']):
                        continue
                    userPuzzleDict[user_key][secondKey] = {activePuzzle :'submitted'}
                elif(event['type'] == 'ws-puzzle_complete'):
                    if (activePuzzle == 'Sandbox'):
                        continue
                    userPuzzleDict[user_key][secondKey] = {activePuzzle :'completed'}
                elif(event['type'] in ['ws-exit_to_menu', 'ws-disconnect', 'ws-login_user']):
                    if (activePuzzle == 'Sandbox'):
                        continue
                    numPuzzles +=1
                    
    userSessionList = []
    for key in userPuzzleDict.keys():
        for sequence in userPuzzleDict[key].keys():
                key_split = sequence.split('~')
                userSessionList.append([key, key_split[1], int(key_split[0]), userPuzzleDict[key][sequence]])

    userSequence = pd.DataFrame(userSessionList, columns=['user', 'session_id', 'sequence', 'task_id'])
    #Recalculate sequence
    mod = []
    for user in userSequence['user'].unique():
            previousAttempt = 1
            n_attempt = 1
            individualDf = userSequence[userSequence['user'] == user]
            for enum, event in individualDf.iterrows():
                if (event['sequence'] != previousAttempt):
                    n_attempt += 1
                previousAttempt = event['sequence']
                event['sequence'] = n_attempt
                mod.append(event)
    modDf = pd.DataFrame(mod, columns=['user', 'session_id', 'sequence', 'task_id'])
    return modDf

In [9]:
pruebas = sequenceBetweenPuzzles(dataEvents, group = 'all')
pruebas

,user,session_id,sequence,task_id
0,56ccce25ead834182d605eff319bfa2c,kivrsjwpdh2g0zie19hpb2gf0tyj1ufy,1,{'1. One Box': 'completed'}
1,56ccce25ead834182d605eff319bfa2c,kivrsjwpdh2g0zie19hpb2gf0tyj1ufy,2,{'2. Separated Boxes': 'completed'}
2,56ccce25ead834182d605eff319bfa2c,kivrsjwpdh2g0zie19hpb2gf0tyj1ufy,3,{'3. Rotate a Pyramid': 'completed'}
3,56ccce25ead834182d605eff319bfa2c,kivrsjwpdh2g0zie19hpb2gf0tyj1ufy,4,{'4. Match Silhouettes': 'submitted'}
4,56ccce25ead834182d605eff319bfa2c,kivrsjwpdh2g0zie19hpb2gf0tyj1ufy,5,{'3. Rotate a Pyramid': 'started'}
...,...,...,...,...
5796,fb504c1409250aaef610c9d9b7306e18,ukhvuu904a395hg0n7soo81kpnq7llym,6,{'6. Stretch a Ramp': 'completed'}
5797,fb504c1409250aaef610c9d9b7306e18,ukhvuu904a395hg0n7soo81kpnq7llym,7,{'7. Max 2 Boxes': 'submitted'}
5798,5cce64bf55190c8816d80129f875b351,dsx2bjuktwqjcizqqa7y1wiumugyjn86,1,{'1. One Box': 'completed'}
5799,5cce64bf55190c8816d80129f875b351,dsx2bjuktwqjcizqqa7y1wiumugyjn86,2,{'2. Separated Boxes': 'started'}


In [10]:
pruebas.to_csv("sequenceBetwennPuzzlesOutput.csv", decimal = ".")